# Basic selection rules {#basic_selection_rules_example}

This example shows the basic usage of selection rules, which enable you
to select elements through geometrical operations and thus to shape
plies. The example only shows the PyACP part of the setup. For more
advanced selection rule usage, see
`advanced_selection_rules_example`{.interpreted-text role="ref"}. For a
complete composite analysis, see
`pymapdl_workflow_example`{.interpreted-text role="ref"}.


# Import modules

Import the standard library and third-party dependencies.


In [ ]:
import pathlib
import tempfile

Import the PyACP dependencies.


In [ ]:
from ansys.acp.core import LinkedSelectionRule, launch_acp
from ansys.acp.core.extras import ExampleKeys, get_example_file

# Start ACP and load the model


Get the example file from the server.


In [ ]:
tempdir = tempfile.TemporaryDirectory()
WORKING_DIR = pathlib.Path(tempdir.name)
input_file = get_example_file(ExampleKeys.MINIMAL_FLAT_PLATE, WORKING_DIR)

Launch the PyACP server and connect to it.


In [ ]:
acp = launch_acp()

Load the model from the input file.


In [ ]:
model = acp.import_model(input_file)
print(model.unit_system)

Visualize the loaded mesh.


In [ ]:
mesh = model.mesh.to_pyvista()
mesh.plot(show_edges=True)

# Create a Parallel Rule


Create parallel selection rule and assign it to the existing ply.


In [ ]:
parallel_rule = model.create_parallel_selection_rule(
    name="parallel_rule",
    origin=(0, 0, 0),
    direction=(1, 0, 0),
    lower_limit=0.005,
    upper_limit=1,
)

modeling_ply = model.modeling_groups["modeling_group"].modeling_plies["ply"]
modeling_ply.selection_rules = [LinkedSelectionRule(parallel_rule)]

Plot the ply thickness.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Create a Cylindrical Rule


Create a cylindrical selection rule and add it to the ply. This will
intersect the two rules.


In [ ]:
cylindrical_rule = model.create_cylindrical_selection_rule(
    name="cylindrical_rule",
    origin=(0.005, 0, 0.005),
    direction=(0, 1, 0),
    radius=0.002,
)

modeling_ply.selection_rules.append(LinkedSelectionRule(cylindrical_rule))

Plot the ply thickness.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Create a Spherical Rule


Create a spherical selection rule and assign it to the ply. Now, only
the spherical rule is active.


In [ ]:
spherical_rule = model.create_spherical_selection_rule(
    name="spherical_rule",
    origin=(0.003, 0, 0.005),
    radius=0.002,
)

modeling_ply.selection_rules = [LinkedSelectionRule(spherical_rule)]

Plot the ply thickness.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)

# Create a Tube Rule


Create a tube selection rule and assign it to the ply. Now, only the
tube rule is active.


In [ ]:
tube_rule = model.create_tube_selection_rule(
    name="spherical_rule",
    # Select the pre-exsting _FIXEDSU edge which is the edge at x=0
    edge_set=model.edge_sets["_FIXEDSU"],
    inner_radius=0.001,
    outer_radius=0.003,
)

modeling_ply.selection_rules = [LinkedSelectionRule(tube_rule)]

Plot the ply thickness.


In [ ]:
model.update()
assert model.elemental_data.thickness is not None
model.elemental_data.thickness.get_pyvista_mesh(mesh=model.mesh).plot(show_edges=True)